<a href="https://colab.research.google.com/github/mariskaiz/TugasML/blob/main/ML2_Ind_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# penginputan dataset
data_path = '/content/smoking_health_data_final.csv'
data = pd.read_csv(data_path)

# tampilkan tabel awal
data.head()

,age,sex,current_smoker,heart_rate,blood_pressure,cigs_per_day,chol
0,54,male,yes,95,110/72,NaN,219.0
1,45,male,yes,64,121/72,NaN,248.0
2,58,male,yes,81,127.5/76,NaN,235.0
3,42,male,yes,90,122.5/80,NaN,225.0
4,42,male,yes,62,119/80,NaN,226.0


In [23]:


#mengatasi missing value dengan mengisinya dengan median
#pengecekan missing value
missing_values = data.isnull().sum()
print("Missing values per column:\n", missing_values)

Missing values per column:
 age                0
sex                0
current_smoker     0
heart_rate         0
blood_pressure     0
cigs_per_day      14
chol               7
dtype: int64


In [24]:
#pengisian median
for column in ['heart_rate', 'cigs_per_day', 'chol']:
    median_value = data[column].median()
    data[column].fillna(median_value, inplace=True)

# pembagian 'blood_pressure' menjadi dua kolom yaitu 'systolic_bp' dan 'diastolic_bp'
data[['systolic_bp', 'diastolic_bp']] = data['blood_pressure'].str.split('/', expand=True).astype(float)

# menghilangkan kolom 'blood_pressure'
data.drop(columns=['blood_pressure'], inplace=True)

# Summarize key statistics untuk setiap variabel
summary_statistics = data.describe().T
summary_statistics['range'] = summary_statistics['max'] - summary_statistics['min']
print("\nSummary Statistics:\n", summary_statistics)


Summary Statistics:
                count        mean        std    min    25%    50%    75%  \
age           3900.0   49.537949   8.559275   32.0   42.0   49.0   56.0   
heart_rate    3900.0   75.688974  12.015376   44.0   68.0   75.0   82.0   
cigs_per_day  3900.0    9.135641  12.028875    0.0    0.0    0.0   20.0   
chol          3900.0  236.591282  44.335649  113.0  206.0  234.0  263.0   
systolic_bp   3900.0  132.395385  21.976335   83.5  117.0  128.0  144.0   
diastolic_bp  3900.0   82.987179  11.918623   48.0   75.0   82.0   90.0   

                max  range  
age            70.0   38.0  
heart_rate    143.0   99.0  
cigs_per_day   70.0   70.0  
chol          696.0  583.0  
systolic_bp   295.0  211.5  
diastolic_bp  142.5   94.5  


In [25]:
#pengecekan data baru
print("jumlah baris dan kolom")
print(data.shape)
print(sorted(data.isna().sum()))

jumlah baris dan kolom
(3900, 8)
[0, 0, 0, 0, 0, 0, 0, 0]


In [26]:
data.head()

,age,sex,current_smoker,heart_rate,cigs_per_day,chol,systolic_bp,diastolic_bp
0,54,male,yes,95,0.0,219.0,110.0,72.0
1,45,male,yes,64,0.0,248.0,121.0,72.0
2,58,male,yes,81,0.0,235.0,127.5,76.0
3,42,male,yes,90,0.0,225.0,122.5,80.0
4,42,male,yes,62,0.0,226.0,119.0,80.0


In [28]:
data.to_csv('data_fix1')

In [30]:


# Fungsi untuk menentukan kondisi kesehatan berdasarkan kriteria tertentu
def tentukan_kondisi(row):
    if row['age'] > 50 and row['systolic_bp'] > 140:
        return 'Hipertensi'
    elif row['cigs_per_day'] > 15:
        return 'Perokok Berat'
    elif row['chol'] > 240:
        return 'Kolesterol Tinggi'
    elif (row['systolic_bp'] > 120 and row['systolic_bp'] < 140) or (row['diastolic_bp'] > 80 and row['diastolic_bp'] < 90):
        return 'Prehipertensi'
    elif (row['age'] > 65) or (row['heart_rate'] < 60 or row['heart_rate'] > 100):
        return 'Tidak Sehat'
    else:
        return 'Sehat'

# Membuat kolom baru "kondisi_kesehatan" dengan menggunakan fungsi di atas
data['Kondisi_Kesehatan'] = data.apply(tentukan_kondisi, axis=1)

# Menampilkan DataFrame hasil
print(data)

      Unnamed: 0  age   sex current_smoker  heart_rate  cigs_per_day   chol  \
0              0   54  male            yes          95           0.0  219.0   
1              1   45  male            yes          64           0.0  248.0   
2              2   58  male            yes          81           0.0  235.0   
3              3   42  male            yes          90           0.0  225.0   
4              4   42  male            yes          62           0.0  226.0   
...          ...  ...   ...            ...         ...           ...    ...   
3895        3895   37  male            yes          88          60.0  254.0   
3896        3896   49  male            yes          70          60.0  213.0   
3897        3897   56  male            yes          70          60.0  246.0   
3898        3898   50  male            yes          85          60.0  340.0   
3899        3899   40  male            yes          98          70.0  210.0   

      systolic_bp  diastolic_bp  Kondisi_Kesehatan 

In [39]:

print(data.columns)

Index(['Unnamed: 0', 'age', 'sex', 'current_smoker', 'heart_rate',
       'cigs_per_day', 'chol', 'systolic_bp', 'diastolic_bp',
       'Kondisi_Kesehatan'],
      dtype='object')


In [41]:
data2 = data.drop('sex', axis=1)
data2.head(10)

,Unnamed: 0,age,current_smoker,heart_rate,cigs_per_day,chol,systolic_bp,diastolic_bp,Kondisi_Kesehatan
0,0,54,yes,95,0.0,219.0,110.0,72.0,Sehat
1,1,45,yes,64,0.0,248.0,121.0,72.0,Kolesterol Tinggi
2,2,58,yes,81,0.0,235.0,127.5,76.0,Prehipertensi
3,3,42,yes,90,0.0,225.0,122.5,80.0,Prehipertensi
4,4,42,yes,62,0.0,226.0,119.0,80.0,Sehat
5,5,57,yes,62,0.0,223.0,107.5,72.5,Sehat
6,6,43,yes,75,0.0,222.0,109.5,69.0,Sehat
7,7,42,yes,66,0.0,196.0,123.0,73.0,Prehipertensi
8,8,37,yes,65,0.0,188.0,123.5,77.0,Prehipertensi
9,9,49,yes,93,0.0,256.0,127.5,81.5,Kolesterol Tinggi


In [42]:
data3 = data2.drop('Unnamed: 0', axis=1)
data3.head(10)

,age,current_smoker,heart_rate,cigs_per_day,chol,systolic_bp,diastolic_bp,Kondisi_Kesehatan
0,54,yes,95,0.0,219.0,110.0,72.0,Sehat
1,45,yes,64,0.0,248.0,121.0,72.0,Kolesterol Tinggi
2,58,yes,81,0.0,235.0,127.5,76.0,Prehipertensi
3,42,yes,90,0.0,225.0,122.5,80.0,Prehipertensi
4,42,yes,62,0.0,226.0,119.0,80.0,Sehat
5,57,yes,62,0.0,223.0,107.5,72.5,Sehat
6,43,yes,75,0.0,222.0,109.5,69.0,Sehat
7,42,yes,66,0.0,196.0,123.0,73.0,Prehipertensi
8,37,yes,65,0.0,188.0,123.5,77.0,Prehipertensi
9,49,yes,93,0.0,256.0,127.5,81.5,Kolesterol Tinggi


In [46]:
data4 = data3.drop('current_smoker', axis=1)
data4.head(10)

,age,heart_rate,cigs_per_day,chol,systolic_bp,diastolic_bp,Kondisi_Kesehatan
0,54,95,0.0,219.0,110.0,72.0,Sehat
1,45,64,0.0,248.0,121.0,72.0,Kolesterol Tinggi
2,58,81,0.0,235.0,127.5,76.0,Prehipertensi
3,42,90,0.0,225.0,122.5,80.0,Prehipertensi
4,42,62,0.0,226.0,119.0,80.0,Sehat
5,57,62,0.0,223.0,107.5,72.5,Sehat
6,43,75,0.0,222.0,109.5,69.0,Sehat
7,42,66,0.0,196.0,123.0,73.0,Prehipertensi
8,37,65,0.0,188.0,123.5,77.0,Prehipertensi
9,49,93,0.0,256.0,127.5,81.5,Kolesterol Tinggi


In [47]:
# Memisahkan variabel target dan prediktor
X = data4.drop('Kondisi_Kesehatan', axis=1)  # Variabel prediktor
y = data4['Kondisi_Kesehatan']  # Variabel target

# Menampilkan variabel prediktor
print("Variabel Prediktor (Features):")
print(X)

# Menampilkan variabel target
print("\nVariabel Target:")
print(y)

Variabel Prediktor (Features):
      age  heart_rate  cigs_per_day   chol  systolic_bp  diastolic_bp
0      54          95           0.0  219.0        110.0          72.0
1      45          64           0.0  248.0        121.0          72.0
2      58          81           0.0  235.0        127.5          76.0
3      42          90           0.0  225.0        122.5          80.0
4      42          62           0.0  226.0        119.0          80.0
...   ...         ...           ...    ...          ...           ...
3895   37          88          60.0  254.0        122.5          82.5
3896   49          70          60.0  213.0        123.0          75.0
3897   56          70          60.0  246.0        125.0          79.0
3898   50          85          60.0  340.0        134.0          95.0
3899   40          98          70.0  210.0        132.0          86.0

[3900 rows x 6 columns]

Variabel Target:
0                   Sehat
1       Kolesterol Tinggi
2           Prehipertensi
3       

In [48]:
# Pembagian data menjadi data pelatihan dan data pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Membangun model Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Melatih model
model.fit(X_train, y_train)

# Prediksi menggunakan data pengujian
predictions = model.predict(X_test)

# Evaluasi model
accuracy = accuracy_score(y_test, predictions)
print("Akurasi:", accuracy)

Akurasi: 0.9910256410256411


In [49]:
# Prediksi menggunakan data pengujian
predictions = model.predict(X_train)

# Evaluasi model
accuracy = accuracy_score(y_train, predictions)
print("Akurasi:", accuracy)

Akurasi: 1.0


In [12]:
print(X_train)

   age  heart_rate  cigs_per_day  chol  systolic_bp  diastolic_bp
4   42          62             0   226          119            80
2   58          81             0   235          127            76
0   65         105            16   248          110            72
3   42          90             0   225          122            80
